In [1]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
from pathlib import Path

CURRENT_DIR = os.getcwd()
print("CURRENT_DIR:", CURRENT_DIR)

BASE_DIR = os.path.abspath(os.path.join(CURRENT_DIR, "..", ".."))
print("BASE_DIR:", BASE_DIR)

DATA_YAML = os.path.join(BASE_DIR, "datasets", "coin", "data.yaml")
print("DATA_YAML:", DATA_YAML)


def remove_background_circular(image, bbox):
    """Отстранува позадина претпоставувајќи дека монетата е кружна"""
    x1, y1, x2, y2 = map(int, bbox)
    cropped = image[y1:y2, x1:x2].copy()

    # Проверка дали сликата е валидна
    if cropped.size == 0 or cropped.shape[0] == 0 or cropped.shape[1] == 0:
        # Врати празна BGRA слика
        return np.zeros((1, 1, 4), dtype=np.uint8)

    gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    circles = cv2.HoughCircles(
        blurred,
        cv2.HOUGH_GRADIENT,
        dp=1,
        minDist=50,
        param1=50,
        param2=30,
        minRadius=int(min(cropped.shape[:2]) * 0.3),
        maxRadius=int(max(cropped.shape[:2]) * 0.6)
    )

    mask = np.zeros(cropped.shape[:2], dtype=np.uint8)

    if circles is not None:
        circles = np.uint16(np.around(circles))
        for circle in circles[0, :1]:
            center = (circle[0], circle[1])
            radius = circle[2]
            cv2.circle(mask, center, radius, 255, -1)
    else:
        h, w = cropped.shape[:2]
        center = (w // 2, h // 2)
        axes = (int(w * 0.45), int(h * 0.45))
        cv2.ellipse(mask, center, axes, 0, 0, 360, 255, -1)

    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.GaussianBlur(mask, (5, 5), 0)

    result = cropped.copy()
    bgra = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
    bgra[:, :, 3] = mask

    return bgra


# Конфигурација
TRAIN_NEW = True  # Промени во False за да користиш постоечки модел
MODEL_PATH = os.path.join(BASE_DIR, "models", "coin_detector_150", "weights", "best.pt")

if TRAIN_NEW:
    print("\n🏋️  ТРЕНИРАЊЕ НА МОДЕЛ ЗА МОНЕТИ СО ПОДОБРЕНИ ПАРАМЕТРИ...")
    model = YOLO('yolov8s.pt')

    results = model.train(
        data=DATA_YAML,
        epochs=150,  # ЗГОЛЕМЕНО: Повеќе епохи за подобро учење
        imgsz=640,
        batch=16,  # ЗГОЛЕМЕНО: Поголем batch за побрзо тренирање
        patience=30,  # Почекај 30 епохи без подобрување пред да запреш
        project=os.path.join(BASE_DIR, "models"),
        name='coin_detector_150',
        device=0,

        # Optimizer параметри
        optimizer="AdamW",
        lr0=0.001,  # Почетна learning rate
        lrf=0.01,  # Финална learning rate
        momentum=0.937,
        weight_decay=0.0005,
        warmup_epochs=3.0,
        warmup_momentum=0.8,

        # Аугментација за подобра генерализација
        hsv_h=0.015,  # Варијација на нијанса
        hsv_s=0.7,    # Варијација на заситеност
        hsv_v=0.4,    # Варијација на светлина
        degrees=15,   # Ротација
        translate=0.1,  # Транслација
        scale=0.5,    # Скалирање
        shear=0.0,    # Искривување
        perspective=0.0,
        flipud=0.5,   # Вертикално превртување
        fliplr=0.5,   # Хоризонтално превртување
        mosaic=1.0,   # Mosaic аугментација
        mixup=0.0,

        # За подобро разликување на слични класи
        cls=0.5,  # Зголемена тежина на класификација
        box=7.5,  # Тежина на bounding box
        dfl=1.5,

        save=True,
        save_period=10,  # Зачувај на секои 10 епохи
        plots=True,
        verbose=True
    )

    print("\n✅ Тренирањето заврши!")
    print(f"📁 Моделот е зачуван во: {os.path.join(BASE_DIR, 'models', 'coin_detector_150', 'weights', 'best.pt')}\n")

    # Прикажи резултати од тренирање
    print("\n📊 РЕЗУЛТАТИ ОД ТРЕНИРАЊЕ:")
    print(f"   Провери ги графиконите во: {os.path.join(BASE_DIR, 'models', 'coin_detector_150')}/")

    model = YOLO(os.path.join(BASE_DIR, "models", "coin_detector_150", "weights", "best.pt"))

else:
    print(f"\n📂 Вчитување на модел од: {MODEL_PATH}")
    try:
        model = YOLO(MODEL_PATH)
        print("✅ Моделот е успешно вчитан!")

        # Прикажи информации за моделот
        print("\n📋 Информации за моделот:")
        print(f"   Класи: {len(model.names)}")
        print(f"   Имиња на класи: {list(model.names.values())}")
        print()

    except Exception as e:
        print(f"❌ Не може да се најде моделот: {MODEL_PATH}")
        print(f"   Грешка: {e}")
        print("\n💡 Опции:")
        print("   • Промени TRAIN_NEW = True за да тренираш нов модел")
        print("   • Качи го твојот .pt фајл и промени го MODEL_PATH")
        print("   • Или користи базичен модел (нема да работи добро за монети)\n")
        model = YOLO('yolov8s.pt')
        print("⚠️  Користам базичен модел - НЕМА ДА ДЕТЕКТИРА ДОБРО!\n")


def test_detection_with_bg_removal(image_path, save_output=True):
    """
    Тестирај детекција на монети со отстранување на позадина

    Args:
        image_path: Патека до сликата
        save_output: Зачувај ли процесирани монети
    """
    if not os.path.exists(image_path):
        print(f"❌ Сликата не постои: {image_path}")
        return None

    print(f"\n🔍 Детектирање на монети во: {image_path}")
    image = cv2.imread(image_path)

    if image is None:
        print(f"❌ Не може да се вчита сликата: {image_path}")
        return None

    # Детектирај монети
    results = model(image, conf=0.25)

    detected_coins = []

    # Обработи секоја детекција
    for idx, result in enumerate(results):
        boxes = result.boxes
        print(f"\n   Пронајдени {len(boxes)} монети:")

        for i, box in enumerate(boxes):
            bbox = box.xyxy[0].cpu().numpy()
            conf = box.conf[0].cpu().numpy()
            cls = int(box.cls[0].cpu().numpy())

            class_name = model.names[cls]
            print(f"   {i+1}. {class_name} (confidence: {conf:.2%})")

            # Отстрани позадина
            coin_bgra = remove_background_circular(image, bbox)
            detected_coins.append({
                'class': class_name,
                'confidence': conf,
                'bbox': bbox,
                'image': coin_bgra
            })

            # Зачувај процесирана монета
            if save_output:
                output_dir = os.path.join(BASE_DIR, "output", "coins")
                os.makedirs(output_dir, exist_ok=True)

                output_path = os.path.join(
                    output_dir,
                    f"coin_{i+1}_{class_name}_{conf:.2f}.png"
                )
                cv2.imwrite(output_path, coin_bgra)
                print(f"      💾 Зачувано: {output_path}")

    return detected_coins


def validate_model():
    """Валидирај модел на validation set"""
    print("\n📊 Валидација на моделот...")
    metrics = model.val()

    print("\n✅ Резултати од валидација:")
    print(f"   mAP50: {metrics.box.map50:.3f}")
    print(f"   mAP50-95: {metrics.box.map:.3f}")
    print(f"   Precision: {metrics.box.mp:.3f}")
    print(f"   Recall: {metrics.box.mr:.3f}")

    return metrics


print("\n🎯 Моделот е подготвен за употреба!")
print("\n💡 Достапни функции:")
print("   • test_detection_with_bg_removal('path/to/image.jpg') - Тестирај со отстранување на позадина")
print("   • validate_model() - Валидирај модел на validation set")
print("   • model.predict('path/to/image.jpg') - Основна предикција\n")

CURRENT_DIR: C:\Users\User5549\Desktop\Marija\MKD-Currency-Detector\CurrencyDetectorApp\yolov8_training\notebooks\train
BASE_DIR: C:\Users\User5549\Desktop\Marija\MKD-Currency-Detector\CurrencyDetectorApp\yolov8_training
DATA_YAML: C:\Users\User5549\Desktop\Marija\MKD-Currency-Detector\CurrencyDetectorApp\yolov8_training\datasets\coin\data.yaml

🏋️  ТРЕНИРАЊЕ НА МОДЕЛ ЗА МОНЕТИ СО ПОДОБРЕНИ ПАРАМЕТРИ...
Ultralytics 8.3.241  Python-3.11.9 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5070, 12227MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\User5549\Desktop\Marija\MKD-Currency-Detector\CurrencyDetectorApp\yolov8_training\datasets\coin\data.yaml, degrees=15, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs